In [104]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init
import random 
import numpy as np

import time

In [105]:
class VanillaRNN(nn.Module):
    def __init__(self, num_input, num_hidden, num_output):
        super(VanillaRNN, self).__init__()
        
        self.num_hidden = num_hidden
        
        self.i2h = nn.Linear(num_input+num_hidden, num_hidden)
        self.h2o = nn.Linear(num_hidden, num_output)
        self.tanh = nn.Tanh()
        
        init.xavier_normal_(self.i2h.weight)
        init.xavier_normal_(self.h2o.weight)
    
    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        
        # Input+Hidden to Hidden
        a = self.i2h(input_combined)
        
        # Activation
        hidden = self.tanh(a)
        
        # Hidden to Output
        output = self.h2o(hidden)
        
        return output, hidden
    
    # Helper to init hidden state
    def initHidden(self):
        return torch.zeros(1, self.num_hidden)

### Load data

In [106]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/Vietnamese.txt', 'data/names/English.txt', 'data/names/Russian.txt', 'data/names/Dutch.txt', 'data/names/French.txt', 'data/names/German.txt', 'data/names/Spanish.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Arabic.txt', 'data/names/Irish.txt', 'data/names/Italian.txt', 'data/names/Japanese.txt', 'data/names/Scottish.txt', 'data/names/Greek.txt', 'data/names/Korean.txt', 'data/names/Portuguese.txt', 'data/names/Polish.txt']


In [107]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def nameToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample(device=None):
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    if device:
        category_tensor = category_tensor.to(computing_device)
        line_tensor = line_tensor.to(computing_device)
    return category, line, category_tensor, line_tensor

print(nameToTensor('Jones').size())

torch.Size([5, 1, 57])


In [111]:
def get_computing_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")
    
LEARNING_RATE = 0.005
INPUT = n_letters
HIDDEN = 128
OUTPUT = len(all_categories)
computing_device = get_computing_device()

n_iters = 100_000
print_every = 5000

current_loss = 0.0

rnn = VanillaRNN(INPUT, HIDDEN, OUTPUT)
rnn.to(computing_device)
criterion = nn.CrossEntropyLoss()

In [ ]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = np.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

def train_one_complete_sample(name_tensor, label_tensor):
    hidden = rnn.initHidden().to(computing_device)
    rnn.zero_grad()
    
    for i in range(name_tensor.size()[0]):
        output, hidden = rnn(name_tensor[i], hidden)
        
    loss = criterion(output, label_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-LEARNING_RATE, p.grad.data)
        
    return output, loss.item()    

for i_iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample(computing_device)
    output, loss = train_one_complete_sample(line_tensor, category_tensor)
    current_loss += loss
    
    if i_iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (i_iter, i_iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

5000 5% (0m 12s) 1.3690 Kau / Vietnamese ✗ (Chinese)
